<a href="https://colab.research.google.com/github/luisosmx/Python_exercises/blob/main/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#################################################################################
#                                                                               #
#  quickbooks_manager.py                                                        #
#                                                                               #
#  Description:                                                                 #
#  This script provides a QuickBooksManager class for interacting with the      #
#  QuickBooks API to manage customers.                                          #
#                                                                               #
#  Input:                                                                       #
#  - client_id:        QuickBooks API client ID.                                #
#  - client_secret:    QuickBooks API client secret.                            #
#  - redirect_uri:     Redirect URI for authentication.                         #
#  - environment:      QuickBooks environment ('sandbox' - 'production').       #
#  - refresh_token:    Refresh token for authentication.                        #
#  - company_id:       QuickBooks company ID.                                   #
#  - family_name:      Family name of the customer to be added.                 #
#  - given_name:       Given name of the customer to be added.                  #
#                                                                               #
#  Flow:                                                                        #
#  Step 1: Authenticate with the QuickBooks API.                                #
#  Step 2: Add a new customer using the provided family_name and given_name.    #
#  Step 3: Print the names of all customers.                                    #
#                                                                               #
#  Output:                                                                      #
#  - New customer ID.                                                           #
#                                                                               #
#################################################################################

#-------------------------Delta Smith Libraries ------------------------------#
#-----------------------------------------------------------------------------#

from deltasmith_libraries.deltasmith_utility import DeltaSmith_Utility
from deltasmith_libraries.configuration_project_name import PROJECT_INFO, CLOUD_FUNCTIONS, SECRETS, BIGQUERY_CONFIG, LOGGER
from deltasmith_apis.apis_configuration_project_name import api_dictionary

#-----------------------------------------------------------------------------#
#----------------------------External Libraries ------------------------------#
from intuitlib.client import AuthClient
from intuitlib.enums import Scopes
from quickbooks import QuickBooks
from quickbooks.objects.customer import Customer
#-----------------------------------------------------------------------------#
#-----------------------------------------------------------------------------#

# region ------------------------Instances------------------------------------#
delta_utilities = DeltaSmith_Utility()
api_connection = APIname_API_Connection()
# endregion

# region ------------------------Variables------------------------------------#

# ----------------------------Project Variables-------------------------------#
project_id = PROJECT_INFO['project_id']
project_name = PROJECT_INFO['project_name']
cloud_function_name = CLOUD_FUNCTIONS['Google_Sheets']

# ------------------------------APIs Variales---------------------------------#
el = api_dictionary['EL']
write_option = BIGQUERY_CONFIG['Append']

# ------------------------------lOGs Variales---------------------------------#
secret_name_deltasmith = SECRETS['DELTASMITH']
table_logs_id = LOGGER['bigquery_conf']['table_id']
type_log_dict = LOGGER["type_log_dict"]['1']
resource_dict = LOGGER["resource_dict"]['1']
resource_name_dict = LOGGER["resource_name_dict"]['2']
destination_project_id_dict = LOGGER["destination_project_id_dict"]['1']
frequency_dict = LOGGER["frequency_dict"]["day"]

# endregion ------------------------------------------------------------------#



def main(request):
    try:
        st = time.time() # Leave this variable as it is

        # region --------------------Code-------------------------------------#

        print('Hello')
        df = 'At the end of the code a dataframe with information processed needs to be created to be sent to get the metadata'

        # endregion ----------------------------------------------------------#

        et = time.time() # Leave this variable as it is

        # region --------------------Metadata-------------------------------------#
        metadata = delta_utilities.map_metadata_cloud_function(
            project_id, project_name, cloud_function_name, df, st, et, el)

        # Filter metadata  -  Make sure that the same information does not exist in bigquery
        df_filter = delta_utilities.filter_existing_metadata(metadata)

        save_metadata = True
        if df_filter.empty is False:
            print("---------------INFO: METADATA ALREADY EXIST IN BIGQUERY TABLE AND WON'T BE SAVED. ----------------------")
        else:
            print(
                f"---------------INFO: METADATA TO SEND: {metadata}----------------------")

            save_metadata = delta_utilities.save_in_bigquery_metadata(metadata,project_id)
        # endregion ----------------------------------------------------------#


        if save_metadata is True:

            severity_dict = LOGGER["severity_dict"]['1']
            description_dict = LOGGER["description_dict"]["INFO"]
            delta_utilities.save_logs(
                severity_dict,
                project_id,
                project_name,
                type_log_dict,
                resource_dict,
                resource_name_dict,
                destination_project_id_dict,
                description_dict,
                frequency_dict,
                secret_name_deltasmith
            )
            print(
                "---------------INFO: SAVING LOGS - SUCCESSFUL ----------------------")
        return "True"
    except Exception as e:
        print(f"An error occurred: {e}")
        severity_dict = LOGGER["severity_dict"]['4']
        description_dict = LOGGER["description_dict"]["ERROR"]
        delta_utilities.save_logs(
            severity_dict,
            project_id,
            project_name,
            type_log_dict,
            resource_dict,
            resource_name_dict,
            destination_project_id_dict,
            description_dict,
            frequency_dict,
            secret_name_deltasmith
        )
        print(
        "---------------INFO: SAVING ERROR LOGS - SUCCESSFUL ----------------------")
        return "False"